<a href="https://colab.research.google.com/github/GuiilhermeOliveira/Projeto_IA_Gemini/blob/main/Chatbot_Cancer_de_Mama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala a biblioteca Google Generative AI
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 34.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai
import textwrap
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# Configura a chave de API do Google Generative AI
GOOGLE_API_KEY="API do Google"
genai.configure(api_key=GOOGLE_API_KEY)
# Define o modelo de embedding a ser usado
model = "models/embedding-001"

In [3]:
# Função para gerar embeddings a partir do título e do texto
def embed_fn(title, text):
  return genai.embed_content(model=model,
                            content=text,
                            title=title,
                            task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [4]:
# Carrega a planilha Excel contendo os dados
file_path = '/content/CancerDeMama.xlsx'  # substitua pelo caminho do seu arquivo
df = pd.read_excel(file_path)
df = pd.DataFrame(df)
# Adiciona uma nova coluna "Embedding" ao DataFrame com os embeddings gerados
df["Embedding"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)

In [5]:
# Função para gerar uma consulta e buscar a resposta mais relevante na base de dados
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                            content=consulta,
                            task_type="RETRIEVAL_QUERY")
  produtos_escalares = np.dot(np.stack(df["Embedding"]), embedding_da_consulta["embedding"])

  indices = np.argmax(produtos_escalares)
  return df.iloc[indices]["Conteudo"]

In [6]:

# Configurações de geração de conteúdo
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
    "top_k": 35,
    "top_p": 0.95,
}
# Configurações de segurança para o modelo generativo
safety_setting = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",

}

In [ ]:

# Define o modelo generativo a ser usado
model_2 = genai.GenerativeModel(model_name="gemini-1.5-pro",
                                generation_config=generation_config,
                                safety_settings=safety_setting)

# Função para converter texto em Markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Função para processar a consulta
def processar_consulta(widget):
    consulta = widget.value.strip()
    if not consulta:
        return

    if consulta.lower() == "sair":
        print("Encerrando a conversa!!!.")
        widget.close()
        return

    try:
        # Gerar consulta e obter resposta
        trecho = gerar_e_buscar_consulta(consulta, df, model)
        prompt = f"Escreva o texto de forma informativa, sem adicionar informações que não façam parte do texto: {trecho}"
        response = model_2.generate_content(prompt)

        # Exibir resposta formatada em Markdown
        display(to_markdown(f"**Resposta**: {response.text}"))

    except Exception as e:
        print(f"Erro ao processar a consulta: {e}")

    # Remover o widget de input atual
    widget.close()

    # Criar um novo input abaixo da resposta
    new_input = widgets.Text(
        value='',
        placeholder='Digite sua próxima pergunta aqui (ou digite "sair" para encerrar a conversa):',
        description='Usuário:',
        disabled=False,
        layout={'width': '60%'}
    )
    new_input.on_submit(processar_consulta)
    display(new_input)

# Widget de input inicial
input_widget = widgets.Text(
    value='',
    placeholder='Digite sua pergunta aqui (ou digite "sair" para encerrar a conversa):',
    description='Usuário:',
    disabled=False,
    layout={'width': '60%'}
)

# Conecta a função ao widget com on_submit
input_widget.on_submit(processar_consulta)

# Exibe o widget inicial
display(input_widget)
